[XTTS Huggingspace](https://huggingface.co/spaces/coqui/xtts)

In [1]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }

%%html
<b>Press play on the music player to keep the tab alive, then run the cell below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title Install XTTS

# https://huggingface.co/spaces/coqui/xtts
# https://docs.coqui.ai/en/dev/models/xtts.html
!pip install pydub
!pip install googletrans==3.1.0a0
!rm -rf /content/TTS
!rm -rf /content/XTTS-v2
%cd /content/
!git clone https://github.com/coqui-ai/TTS/
%cd /content/TTS
!make system-deps
!make install

%cd /content/
!git lfs install
!git clone https://huggingface.co/coqui/XTTS-v2
!pip install numpy==1.26.2
!pip install tensorflow==2.12.0
from IPython.display import clear_output
clear_output()
#@title Restart Runtime
import time
time.sleep(6)
import os
os.kill(os.getpid(), 9)

In [5]:
#@title Utils

def fix(text, max_chars=100):
    text = text.strip()
    chunks = []
    punctuations = ['!', '?', '#', '$', '%', '&', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
    joining_words = ["I", "I'm", 'and', 'or', 'but', 'nor', 'for', 'so', 'yet', 'after', 'although', 'because', 'before', 'if', 'once', 'since', 'though', 'unless', 'until', 'when', 'where', 'while']
    # Check for punctuations
    for line in text.split("\n"):
        modified_line = line.strip()
        if len(modified_line) > 1:
            current_chunk = ''
            for char in modified_line:
                if char in punctuations:
                    chunks.append(current_chunk + char)
                    current_chunk = ''
                else:
                    current_chunk += char
            if current_chunk:
                chunks.append(current_chunk)
        else:
            chunks.append(modified_line)


    final_chunks = []
    for line in chunks:
        if len(line) > max_chars:
            # If the line has more than max_words, split it based on joining words
            current_chunk = ''
            for word in line.split():
                if word in joining_words:
                    if current_chunk:
                        final_chunks.append(current_chunk)
                    final_chunks.append(word)
                    current_chunk = ''
                else:
                    current_chunk += word + ' '
            if current_chunk:
                final_chunks.append(current_chunk.strip())
        else:
            final_chunks.append(line)

    return final_chunks

def merge_single_elements(input_list):
    result_list = []
    temp_list = []

    for element in input_list:
        # If temp_list is not empty and the last element in temp_list has 1 or 2 words
        if temp_list and len(temp_list[-1].split()) <= 2:
            # Merge the current element with the last element in temp_list
            temp_list[-1] += ' ' + element
        else:
            # If the current element has more than 2 words, add it directly to temp_list
            temp_list.append(element)

    # Handle edge cases at the beginning and end of the list
    if len(temp_list) >= 2 and len(temp_list[0].split()) <= 2:
        # Merge the first element with the second element
        temp_list[0] += ' ' + temp_list[1]
        # Remove the second element
        temp_list.pop(1)
    elif len(temp_list) == 1 and len(temp_list[0].split()) <= 2:
        # If there is only one element with 1 or 2 words, return it
        return [temp_list[0].strip()]

    # Handle the case where the last element is a single word
    if len(temp_list[-1].split()) == 1:
        # Merge it with the preceding element
        temp_list[-2] += ' ' + temp_list[-1]
        # Remove the last element
        temp_list.pop()

    # Clean and filter the modified list
    filtered_list = [element.strip() for element in temp_list if element.strip() and element.strip() != "'" and element.strip() != '"']
    modified_list = [
        i.replace("'", "").strip() if i.strip().startswith("'") or i.strip().endswith("'") else i.strip()
        for i in filtered_list
    ]

    return modified_list

def get_chunks(text,full_stop):
  if full_stop:
    temp_list = text.strip().split(".")
    filtered_list = [element.strip() + '.' for element in temp_list if element.strip() and element.strip() != "'" and element.strip() != '"']
    return filtered_list
  else:
    output_list = merge_single_elements(fix(text, max_chars=200))
    return output_list

# Enter_text = "The old clock tower in the heart of Whisperwind hummed with secrets. Its mossy face, etched with forgotten time, stared down upon Elara, a young woman with eyes the color of twilight and hair like spun moonlight. Tonight, under the cloak of a velvet sky, she stood poised on the precipice of adventure.  Elara wasn't a princess locked in a tower, nor a damsel in distress. She was a mapmaker, her fingers nimble with ink and compass, her heart wild with the yearning to chase uncharted lines across the world. But Whisperwind, her sleepy village nestled between rolling hills, held her captive with its invisible chains of tradition.  Tonight, however, the chains loosened. A cryptic note, left on her doorstep by a shadowy figure, whispered of a hidden map, a key to a lost city whispered in ancient legends. The city, they said, pulsed with forgotten magic, its streets paved with dreams and its towers kissed by stardust.  Elara, her pulse quickening like a hummingbird's wings, knew this was her chance. With a knapsack filled with inkwells and star charts, she slipped into the night, the old clock tower watching as she became a wisp of smoke against the canvas of stars.  Her journey led her through whispering forests and dandelion-dusted meadows, past slumbering giants carved from stone and rivers that sang forgotten lullabies. Along the way, she met companions \u2013 a mischievous pixie with wings of moonlight, a grizzled mountain goat with eyes like polished obsidian, and a wise old tree whose roots pulsed with the rhythm of the earth.  Each companion, with their quirks and riddles, revealed a fragment of the map. The pixie led her through tangled paths, the goat climbed treacherous cliffs, and the tree whispered forgotten words that unlocked hidden doors. As Elara pieced together the map, the city grew closer, its silhouette shimmering on the horizon like a mirage.  Finally, they stood before a forgotten gate, overgrown with vines and humming with an ancient energy. Elara, her heart pounding a tribal rhythm, traced the map's final line on the gate \u2013 a constellation etched in moonlight. With a groan and a sigh, the gate creaked open, revealing a city bathed in ethereal light.  The lost city was a tapestry woven from stardust and dreams. Towers kissed the clouds, their windows glowing with forgotten knowledge. Streets, paved with wishes, whispered stories of the past. And in the heart of it all, a library carved from living crystal held the map, not to a city, but to the infinite possibilities within herself.  Elara, no longer a mapmaker of sleepy villages, but a cartographer of the soul, returned to Whisperwind, not with riches, but with a map etched in her heart. And as she shared her tale, the village, once bound by tradition, stirred with a newfound hunger for adventure. For Elara had shown them that the greatest journeys aren't always outward, but the ones we take within ourselves.  The old clock tower, no longer a silent sentinel, hummed with a different tune now \u2013 a song of possibilities, a melody of dreams waiting to take flight. And under the velvet sky, Elara, the mapmaker with eyes like twilight, smiled, knowing that her greatest adventure had just begun."  # @param {type: "string"}
# full_stop = True # @param {type:"boolean"}

# get_chunks(Enter_text,False)
from googletrans import Translator

def translate_text(text, target_language='en'):
    translator = Translator()
    translation = translator.translate(text, dest=target_language)
    return translation.text

def xtts(Enter_text,save_path,Original_voice_path,language,translate_text_flag):
  if translate_text_flag==True:
    text=translate_text(Enter_text, target_language=language)
  else:
    text=Enter_text
  tts.tts_to_file(text=text,
                file_path=save_path,
                speaker_wav=Original_voice_path,
                language=language)
  return save_path

from pydub import AudioSegment

def merge_audio_files(file_paths, output_path):
    # Create an empty AudioSegment object to store the merged audio
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through the list of audio file paths and append each to the merged_audio
    for file_path in file_paths:
        audio = AudioSegment.from_file(file_path)
        merged_audio += audio

    # Export the merged audio to the specified output_path
    merged_audio.export(output_path, format="wav")
#import tts
from TTS.api import TTS
tts = TTS(model_path="/content/XTTS-v2/", config_path="/content/XTTS-v2/config.json", progress_bar=True, gpu=True)

In [19]:
#@title Upload 6s Audio clip
from google.colab import files
uploaded = files.upload()
upload_list=[]
for fn in uploaded.keys():
  upload_list.append(os.getcwd()+'/'+fn)
from IPython.display import clear_output
clear_output()
upload_list[-1]

'/content/ronaldot.wav'

In [21]:
import os
import shutil

text = "hey guys what are you doing ?"  # @param {type: "string"}

Real_Human_Voice="/content/ronaldot.wav" # @param {type: "string"}
save_path = '/content/clone.wav'  # @param {type: "string"}


def voice_clone(text,voice_name,save_path):
  current=os.getcwd()
  language = "ja" # @param ["en", "hi","ja","zh-cn","ko", "es", "fr", "de", "it", "pt", "pl", "tr", "ru", "nl", "cs", "ar", "zh-cn",  "hu"]
  Split_sentences_by_full_stop = True # @param {type:"boolean"}
  translate_text_flag  = True # @param {type:"boolean"}

  Original_voice_path=voice_name
  list_of_text=get_chunks(text,True)
  if os.path.exists("/content/temp_clone_tts/"):
    shutil.rmtree("/content/temp_clone_tts/")
  os.mkdir("/content/temp_clone_tts/")
  clone_chunk=[]
  for index, text in enumerate(list_of_text):
    print( index, text)
    temp_save_path=f"/content/temp_clone_tts/{index}.wav"
    xtts(text,temp_save_path,Original_voice_path,language,translate_text_flag)
    if os.path.exists(temp_save_path):
      clone_chunk.append(temp_save_path)
  merge_audio_files(clone_chunk, save_path)
  os.chdir(current)


voice_clone(text,Real_Human_Voice,save_path)
from IPython.display import Audio
Audio(save_path, autoplay=True)

0 hey guys what are you doing ?.
 > Text splitted to sentences.
['やあ、みんな何してるの？']
 > Processing time: 1.1461529731750488
 > Real-time factor: 0.4366994929933271


In [ ]:
#@title Download clone voice

from google.colab import files
files.download(save_path)